In [7]:
import tensorflow as tf
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

In [2]:
movie_metadata_path = 'the-movies-dataset/movies_metadata.csv'
movie_metadata = pd.read_csv(movie_metadata_path)

/Users/yangpei/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
rating_path = 'the-movies-dataset/ratings_small.csv'
rating = pd.read_csv(rating_path)
user_ids = rating['userId']
movie_ids = rating['movieId']
ratings = rating['rating']

# Reindex the movie ids
movie_id_reindex = {}
for movie_id in movie_ids:
    if movie_id not in movie_id_reindex:
        idx = len(movie_id_reindex)
        movie_id_reindex[movie_id] = idx

training_reindex = []
for user_id, movie_id, rating in zip(user_ids, movie_ids, ratings):
    training_reindex.append(np.array([user_id, movie_id_reindex[movie_id], rating]))
training_reindex = np.array(training_reindex)

print(len(movie_id_reindex))

9066


In [95]:
NUM_MOVIE = 10000
NUM_USER = 700
EMBEDDING_DIM = 32
HIDDEN1_DIM = 128
HIDDEN2_DIM = 64

tf.reset_default_graph()

# Build the model
movie_id = tf.placeholder(tf.int32, shape=[None], name='movie_id')
user_id = tf.placeholder(tf.int32, shape=[None], name='user_id')
rating = tf.placeholder(tf.float32, shape=[None], name='rating')

movie_embeddings = tf.get_variable('movie_embeddings', [NUM_MOVIE, EMBEDDING_DIM], initializer=tf.truncated_normal_initializer)
user_embeddings = tf.get_variable('user_embeddings', [NUM_USER, EMBEDDING_DIM], initializer=tf.truncated_normal_initializer)

embedded_movie_ids = tf.gather(movie_embeddings, movie_id)
embedded_user_ids = tf.gather(user_embeddings, user_id)
embeddings = tf.concat([embedded_movie_ids, embedded_user_ids], 1)

W1 = tf.get_variable('W1', [2*EMBEDDING_DIM, HIDDEN1_DIM], initializer=tf.truncated_normal_initializer)
b1 = tf.get_variable('b1', [HIDDEN1_DIM], initializer=tf.constant_initializer(0.1))

relu1 = tf.nn.relu(tf.matmul(embeddings, W1) + b1)

W2 = tf.get_variable('W2', [HIDDEN1_DIM, HIDDEN2_DIM], initializer=tf.truncated_normal_initializer)
b2 = tf.get_variable('b2', [HIDDEN2_DIM], initializer=tf.constant_initializer(0.1))

relu2 = tf.nn.relu(tf.matmul(relu1, W2) + b2)

W3 = tf.get_variable('W3', [HIDDEN2_DIM, 1], initializer=tf.truncated_normal_initializer)
b3 = tf.get_variable('b3', [1], initializer=tf.constant_initializer(0.1))

out = tf.matmul(relu2, W3) + b3

loss = tf.reduce_mean(tf.squared_difference(out, rating))
train = tf.train.RMSPropOptimizer(1e-2).minimize(loss)

In [96]:
import math

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

num_data = training_reindex.shape[0]
num_training = math.ceil(num_data * 0.8)
idx = np.arange(num_data)
np.random.shuffle(idx)
training = training_reindex[idx[:num_training], :]
testing = training_reindex[idx[num_training: ], :]

NUM_TRAINING_STEP = 2000
BATCH_SIZE = 500
for i in range(0, NUM_TRAINING_STEP):
    batch = training[np.random.choice(num_training, BATCH_SIZE), :]
    sess.run(train, feed_dict={movie_id: batch[:, 1].astype(int), user_id: batch[:, 0].astype(int), rating: batch[:, 2]})
    
    if i % 100 == 0:
        print(sess.run(loss, feed_dict={movie_id: batch[:, 1].astype(int), user_id: batch[:, 0].astype(int), rating: batch[:, 2]}))

84336.0
2882.14
412.091
258.388
59.0594
28.8527
3.70165
1.55997
1.40887
1.24696
1.0738
1.12769
1.17661
1.37838
1.2007
1.08462
1.23563
1.08515
1.00767
1.16868


In [19]:
from deep_matrix_factorization import DeepMatrixFactorization
import math

num_data = training_reindex.shape[0]
num_training = math.ceil(num_data * 0.8)
idx = np.arange(num_data)
np.random.shuffle(idx)
training = training_reindex[idx[:num_training], :]
testing = training_reindex[idx[num_training: ], :]

dmf = DeepMatrixFactorization(700, 10000, 32, [128, 64])
dmf.learn(training[:, 0], training[:, 1], training[:, 2], 2000, 500)

60947.6
2168.02
514.833
57.4434
9.64513
2.32309
1.45732
1.58789
1.38947
1.11918
1.31885
1.26561
1.40568
1.34647
1.03703
1.18125
1.22656
1.28716
1.22663
1.10302


In [27]:
res = dmf.test(testing[:, 0], testing[:, 1], testing[:, 2])

In [29]:
type(res)

numpy.float32

In [25]:
from matrix_factorization import MatrixFactorization

mf = MatrixFactorization(700, 10000, 32)
mf.learn(training[:, 0], training[:, 1], training[:, 2], 2000, 500)

33.578
27.0607
20.8241
16.8298
16.9472
13.5734
13.0469
10.9891
8.37956
7.16215
6.96553
5.18535
4.35638
4.82083
3.90566
3.89303
3.20582
2.8931
3.03369
2.80142


In [26]:
print(mf.test(testing[:, 0], testing[:, 1], testing[:, 2]))

6.20552
